In [1]:
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [2]:
jet_stream_data = json.load(open("jet_stream_data.json"))

In [3]:
data_dict = dict()
for wind_component_data in jet_stream_data:
    key_name = wind_component_data["header"]["parameterNumberName"]
    key_value = pd.Series(wind_component_data["data"])
    data_dict[key_name] = key_value

In [4]:
jet_stream_dataframe = pd.DataFrame(data_dict)

In [5]:
longitudes = list()
for i in range(181):
    longitudes.extend(list(range(360)))

In [6]:
latitudes = list()
for i in range(90, -91, -1):
    latitudes.extend(list([i]*360))

In [7]:
assert len(latitudes) == len(longitudes)
assert len(latitudes) == len(jet_stream_dataframe)

In [8]:
jet_stream_dataframe["lng"] = longitudes
jet_stream_dataframe["lat"] = latitudes

In [9]:
# columns used for constructing geojson object
features = jet_stream_dataframe.apply(
    lambda row: Feature(geometry=Point((float(row['lng']), float(row['lat'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = jet_stream_dataframe.drop(['lng', 'lat'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

In [10]:
with open('jet_stream_data.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)